<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# Instrumenting AWS Bedrock client with OpenInference and Phoenix

In this tutorial we will trace model calls to AWS Bedrock using OpenInference. The OpenInference Bedrock tracer instruments the Python `boto3` library, so all `invoke_model` calls will automatically generate traces that can be sent to Phoenix.

ℹ️ This notebook requires a valid AWS configuration and access to AWS Bedrock and the `claude-v2` model from Anthropic & an OpenAI API key for LLM as a Judge Evaluation.

## 1. Install dependencies and set up OpenTelemetry tracer

First install dependencies

In [1]:
%pip install arize-phoenix boto3 openinference-instrumentation-bedrock

Note: you may need to restart the kernel to use updated packages.


Import libraries

In [2]:
import json

The following env variables will allow you to connect to a cloud instance of Arize Phoenix. You can get your API key and endpoint on setting page of your [Phoenix Cloud](https://app.phoenix.arize.com) account.

If you'd prefer to self-host Phoenix, please see [instructions for self-hosting](https://arize.com/docs/phoenix/deployment). The Cloud and Self-hosted versions are functionally identical.

In [ ]:
import os
from getpass import getpass

import boto3
from openinference.instrumentation.bedrock import BedrockInstrumentor

import phoenix as px
from phoenix.otel import SimpleSpanProcessor, register

if not (phoenix_endpoint := os.getenv("PHOENIX_COLLECTOR_ENDPOINT")):
    phoenix_endpoint = getpass("🔑 Enter your Phoenix Collector Endpoint: ")
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = phoenix_endpoint

if not (phoenix_api_key := os.getenv("PHOENIX_API_KEY")):
    phoenix_api_key = getpass("🔑 Enter your Phoenix API key: ")
os.environ["PHOENIX_API_KEY"] = phoenix_api_key

Here we're configuring the OpenTelemetry tracer by adding two SpanProcessors. The first SpanProcessor will simply print all traces received from OpenInference instrumentation to the console. The second will export traces to Phoenix so they can be collected and viewed.

In [4]:
tracer_provider = register(project_name="bedrock-tracing-and-evals-tutorial", auto_instrument=True)
tracer_provider.add_span_processor(
    SimpleSpanProcessor(endpoint=os.getenv("PHOENIX_COLLECTOR_ENDPOINT"))
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: bedrock-tracing-and-evals-tutorial
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [9]:
# Alternative: Prompt for AWS credentials if not already set
# Uncomment the code below to be prompted for credentials interactively

if not os.getenv("AWS_ACCESS_KEY_ID"):
    os.environ["AWS_ACCESS_KEY_ID"] = getpass("🔑 Enter your AWS Access Key ID: ")
if not os.getenv("AWS_SECRET_ACCESS_KEY"):
    os.environ["AWS_SECRET_ACCESS_KEY"] = getpass("🔑 Enter your AWS Secret Access Key: ")
# if not os.getenv("AWS_DEFAULT_REGION"):
#     os.environ["AWS_DEFAULT_REGION"] = input("🌍 Enter your AWS Region (default: us-west-2): ") or "us-west-2"

## 2. Instrumenting Bedrock clients

Now, let's create a `boto3` session. This initiates a configured environment for interacting with AWS services. If you haven't yet configured `boto3` to use your credentials, please refer to the [official documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html). Or, if you have the AWS CLI, run `aws configure` from your terminal.

In [10]:
session = boto3.session.Session()

Clients created using this session configuration are currently uninstrumented. We'll make one for comparison.

In [11]:
uninstrumented_client = session.client("bedrock-runtime", region_name="us-west-2")

Now we instrument Bedrock with our OpenInference instrumentor. All Bedrock clients created after this call will automatically produce traces when calling `invoke_model`.

In [12]:
BedrockInstrumentor().instrument(skip_dep_check=True)
instrumented_client = session.client("bedrock-runtime", region_name="us-west-2")

Attempting to instrument while already instrumented


## 3. Calling the LLM and viewing OpenInference traces

Calling `invoke_model` using the `uninstrumented_client` will produce no traces, but will show the output from the LLM.

In [13]:
prompt = b"""{"prompt": "Human: What is the 3rd month of the year in alphabetical order? Assistant:", "max_tokens_to_sample": 1024}"""
response = uninstrumented_client.invoke_model(modelId="anthropic.claude-v2:1", body=prompt)
response_body = json.loads(response.get("body").read())
print(response_body["completion"])

Setting attribute on ended span.
Calling end() on an ended span.


ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the InvokeModel operation: This model version has reached the end of its life. Please refer to the AWS documentation for more details.

LLM calls using the `instrumented_client` will print traces to the console! By configuring the `SpanProcessor` to export to a different OpenTelemetry collector, your OpenInference spans can be collected and analyzed to better understand the behavior of your LLM application.

In [ ]:
response = instrumented_client.invoke_model(modelId="anthropic.claude-v2:1", body=prompt)
response_body = json.loads(response.get("body").read())
print(response_body["completion"])

## 4. Collect all your Traces & Data

Use the `instrumented_client` to collect all your traces; This example uses a set of trivia questions.

In [ ]:
trivia_questions = [
    "What is the only U.S. state that starts with two vowels?",
    "What is the 3rd month of the year in alphabetical order?",
    "What is the capital of Mongolia?",
    "How many minutes are there in a leap year?",
    "If a train leaves New York at 3 PM traveling west at 60 mph, and another leaves Chicago at 4 PM traveling east at 80 mph, at what time will they meet?",
    "Which element has the chemical symbol 'Fe'?",
    "What five-letter word becomes shorter when you add two letters to it?",
    "What country has won the most FIFA World Cups?",
    "If today is Wednesday, what day of the week will it be 100 days from now?",
    "A farmer has 17 sheep and all but 9 run away. How many does he have left?",
]

for i, question in enumerate(trivia_questions, start=1):
    prompt_str = f"""
{{
    "prompt": "Human: {question} Assistant:",
    "max_tokens_to_sample": 300
}}
"""
    response = instrumented_client.invoke_model(
        modelId="anthropic.claude-v2:1",
        body=prompt_str.encode("utf-8"),
        contentType="application/json",
        accept="application/json",
    )

    response_body = json.loads(response.get("body").read())
    print(f"Q{i}: {question}")
    print(f"A{i}: {response_body['completion'].strip()}\n{'-' * 60}")

## 5. Setup & Run your Eval

After importing your traces as a dataframe, modify your columns to fit into your eval template.

In [1]:
qa_template = """You are given a question and an answer. You must determine whether the
given answer correctly answers the question. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {Question}
    ************
    [Answer]: {Answer}
    [END DATA]
Your response must be a single word, either "correct" or "incorrect",
and should not contain any text or characters aside from that word.
"correct" means that the question is correctly and fully answered by the answer.
"incorrect" means that the question is not correctly or only partially answered by the
answer."""

In [ ]:
spans_df = px.Client().get_spans_dataframe(project_name="bedrock-tracing-and-evals-tutorial")
spans_df.head()

In [ ]:
eval_df = spans_df[["context.span_id", "attributes.input.value", "attributes.output.value"]].copy()
eval_df.set_index("context.span_id", inplace=True)
eval_df.head()

In [ ]:
evals_copy = eval_df.copy()
evals_copy["attributes.input.value"] = (
    evals_copy["attributes.input.value"]
    .str.replace(r"^Human: ", "", regex=True)
    .str.replace(r"Assistant:$", "", regex=True)
)

evals_copy = evals_copy.rename(
    columns={"attributes.input.value": "Question", "attributes.output.value": "Answer"}
)
evals_copy.head()

In [ ]:
from openinference.instrumentation import suppress_tracing

from phoenix.evals import (
    async_evaluate_dataframe,
    create_classifier,
)
from phoenix.evals.llm import LLM

llm = LLM(provider="openai", model="gpt-4")

qa_correctness_eval = create_classifier(
    name="qa_correctness",
    prompt_template=qa_template,
    llm=llm,
    choices={"correct": 1.0, "incorrect": 0.0},
)
with suppress_tracing():
    Q_and_A_classifications = await async_evaluate_dataframe(
        dataframe=evals_copy, evaluators=[qa_correctness_eval]
    )
Q_and_A_classifications.head()

## 6. Log your traces into Phoenix

In [ ]:
from phoenix.evals.utils import to_annotation_dataframe

relevancy_eval_df = to_annotation_dataframe(dataframe=Q_and_A_classifications)

await px.Client().annotations.log_span_annotations_dataframe(
    dataframe=relevancy_eval_df,
    annotator_kind="LLM",
)

![GIF showcasing what the phoenix UI will look like with all the traces & eval](https://storage.googleapis.com/arize-phoenix-assets/assets/gifs/bedrock_tracing_eval_medium.gif)

More information about our instrumentation integrations, OpenInference can be found in our [documentation](https://arize.com/docs/phoenix/references/openinference)